# Goal: Look at coupling index from CESM1 amip simulations for comparison to CESM2. 
<br>
<b>Author:</b> Meg Fowler <br>
<b>Date:</b> 27 Aug 2020 
<br><br>
<b>Short Description:</b> Ultimate goal is to determine if CLUBB makes a big difference in the coupling metrics. Should be able to tell that (at least roughly) by first looking at a CESM1 amip simulation compared with the CESM2 run analyzed in <i>Coupling_Index.ipynb</i>.
<br>


In [1]:
# Import libraries 
import comet as cm 
import numpy as np 
import xarray as xr 
import pickle
import pandas as pd
import datetime 

# Plotting utils 
import matplotlib.pyplot as plt 
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.util


## Read in data
<u>Note:</u> Pressure level of the LCL height (in hPa above ground) needs to be computed with separate script for the sake of memory. 

In [ ]:
# Set directories and file names (build up with various parts)
cesm1dir   = '/Users/meganfowler/Documents/NCAR/Analysis/Coupling_initial/data/CESM1/'

startCLM = 'cam5.1_amip_1d_002.clm2.h1.'
startCAM = 'cam5.1_amip_1d_002.cam2.h1.'

endCLM = '_soilWater.nc'
endFLX = '_sfcFluxes.nc'

timeName = ['1979-1989', '1990-2004']


In [ ]:
# Read in test file to get lat/lon 
fileName = cesm1dir+startCLM+timeName[0]+endCLM
testDF   = xr.open_dataset(fileName, decode_times=True)

# Get lat and lon 
lat = testDF.lat.values
lon = testDF.lon.values 


In [ ]:
# Read in surface fluxes along

for iT in range(len(timeName)):

    # --- Compile surface fluxes into singe large arrays in time 

    # Latent & sensible heat fluxes
    flxFile       = cesm1dir+startCAM+timeName[0]+endFLX
    flxDF         = xr.open_dataset(lhFile,decode_times=True) 
    flxDF['time'] = flxDF.indexes['time'].to_datetimeindex()

    # Soil moisture 
    soilFile       = cesm1dir+startCLM+timeName[0]+endCLM
    soilDF         = xr.open_dataset(soilFile,decode_times=True)
    soilDF['time'] = soilDF.indexes['time'].to_datetimeindex()
    
    
    if iT==0:
        flux_full = flxDF 
        soil_full = shDF
    else: 
        flux_full = xr.concat([flux_full, flxDF], dim="time")
        soil_full = xr.concat([soil_full, shDF], dim="time")


    print('Done with ', timeName_flx[iT])



In [ ]:
# Load pressure level of LCL (computed separately in Get_LCLpressure.py and saved as pickle file)
lclPath = '/Users/meganfowler/Documents/NCAR/Analysis/Coupling_initial/Coupling_CAM6CLM5/processed_data/LCL-pressure-HeightAboveGround_CESM1.p'
Plcl    = pickle.load( open( lclPath, "rb" ) )


In [ ]:
# To be able to easily access years, months, days - use Pandas 
dates    = pd.DatetimeIndex(soilM10_full['time'].values)


## Sanity check: do variables seem reasonable?

In [ ]:
# Map out average pressure level of the LCL (in terms of height above sfc)

fig=plt.figure(figsize=(8, 5), dpi= 80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111, projection=ccrs.Robinson())
ax.set_global()

# Add cyclic point (where lon goes from 360˚ to 0˚) -- othwerwise, blank line 
Plcl_cyclic, lon_cyclic = cartopy.util.add_cyclic_point(np.squeeze(np.nanmean(Plcl,0)), lon)

# Make plot
contour_plot = ax.pcolormesh(lon_cyclic,lat,Plcl_cyclic,transform=ccrs.PlateCarree())
ax.plot(lon_cyclic[206],lat[135],'r*',markersize=5,transform=ccrs.PlateCarree())   # Point for sanity check
ax.coastlines()
cbar = plt.colorbar(contour_plot, label='(hPa)',orientation='horizontal')
ax.set_title('Pressure Level of LCL (time average)', fontsize=14)


In [ ]:
# Plot a timeseries of variables 

fig, ax1 = plt.subplots(1,1, figsize=(8, 5), facecolor='w', edgecolor='k')

ax1.plot(dates, flux_full.SHFLX[:,135,206],'k-')
ax1.set_ylabel('SHFLX (W/m2)')
ax1.set_xlabel('Time')
ax1.legend(['SHF'],loc='upper left',fontsize=12)
ax1.set_title('Sanity check: SHF & Plcl [midwest point]', fontsize=14)
ax1.axhline(0,color='gray')

ax2 = ax1.twinx()
ax2.plot(dates, Plcl[:,135,206], 'r-')
ax2.set_ylabel('Plcl height above sfc (hPa)')
ax2.legend(['Plcl'], loc='upper right',fontsize=12)

ax1.set_xlim([datetime.date(1990,6,1), datetime.date(1990,7,1)])


In [ ]:
# How about latent heat flux? 

fig=plt.figure(figsize=(11, 7), dpi= 80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111, projection=ccrs.Robinson())
ax.set_global()

# Add cyclic point (where lon goes from 360˚ to 0˚) -- othwerwise, blank line 
LHavg_cyclic, lon_cyclic = cartopy.util.add_cyclic_point(np.squeeze(np.nanmean(flux_full.LHFLX,0)), lon)

# Make plot
contour_plot = ax.pcolormesh(lon_cyclic,lat,LHavg_cyclic,transform=ccrs.PlateCarree())
ax.coastlines()
cbar = plt.colorbar(contour_plot, label='[W/m2]',orientation='horizontal')
ax.set_title('LHFLX (time mean, 1979-2004)', fontsize=14)


## Compute coupling indices 